In [1]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [2]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_fixed.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [3]:
# building model from URDF, model is an object that includes kinematic and ineratia params
model = pin.buildModelFromUrdf(urdf_filename)

visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)  # very important

collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)

geom_data = pin.GeometryData(collision_model)


# data is an object that holds values that are a result of computation
data = model.createData()

# print('standard model: dim=' + str(len(model.joints)))
# for jn in model.joints:
#     print(jn)
# print('-' * 30)


# # Create a list of joints to lock
# jointsToLock = ['right_foot_shin', 'right_shin_thigh', 'right_thigh_body']
 
# # Get the ID of all existing joints
# jointsToLockIDs = []
# for jn in jointsToLock:
#     if model.existJointName(jn):
#         jointsToLockIDs.append(model.getJointId(jn))
#     else:
#         print('Warning: joint ' + str(jn) + ' does not belong to the model!')

# print(jointsToLockIDs)
 
# geom_models = [visual_model, collision_model]
# model_reduced, geometric_models_reduced = pin.buildReducedModel(
#     model,
#     list_of_geom_models=geom_models,
#     list_of_joints_to_lock=jointsToLockIDs,
#     reference_configuration=pin.neutral(model))
# # geometric_models_reduced is a list, ordered as the passed variable "geom_models" so:
# visual_model_reduced, collision_model_reduced = geometric_models_reduced[
#     0], geometric_models_reduced[1]

In [4]:
visualizer = MeshcatVisualizer(model, collision_model, visual_model)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
q = pin.neutral(model)
print(q)
eps = 1e-4
DT = 1e-2
damp = 1e-2
IT_MAX=2000
JOINT_ID = 6

oMdes = pin.SE3(np.eye(3), np.array([0.092, 0.02, 0.020])) #think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
visualizer.viewer.jupyter_cell()

In [7]:
while True:
    pin.forwardKinematics(model, data, q)
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model, data, q, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(model, q, q_dot * DT)
    
    #Collision check
    pin.updateGeometryPlacements(
        model, data, collision_model, geom_data, q
    )  # updates geom_data by reference

    # an active pair is a pair of bodies in a joint considered for collision
    collision_detected = pin.computeCollisions(
        model, data, collision_model, geom_data, q, True
    )  # this returns data,geom_data. polymorphic function

    if not collision_detected:
        for idx in range(model.nq):
            # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
            q[idx] = max(
                model.lowerPositionLimit[idx],
                min(q[idx], model.upperPositionLimit[idx]),
            )

        visualizer.display(q)
    print(norm_err)
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)


3.144604045647437
3.1360519231172925
3.1269533737690476
3.11611486703736
3.1024990603760916
3.0854263634361265
3.064901955678353
3.0416654849036036
3.0199440209169857
3.023186449285439
3.027159279445383
3.031492966492431
3.035993394330629
3.0405709066880404
3.045144001550926
3.0496403209391203
3.0539976468878534
3.0581647009449
3.062101563370069
3.065779587005457
3.069180768900564
3.0722966375270446
3.075126789050427
3.077677244707533
3.079958799193337
3.081985496559277
3.083773321483524
3.085339145097537
3.0866999256751675
3.08787213912263
3.0888714014605685
3.089712242265231
3.0904079907336235
3.090970741725502
3.091411375704346
3.0917396127677264
3.091964086341412
3.0920924264178473
3.0921313455268806
3.0920867230461715
3.091963685195549
3.0917666792544463
3.0914995413419812
3.0911655576314
3.0907675191826396
3.0903077707729434
3.089788254201375
3.0892105465757598
3.088575894102465
3.0878852418647376
3.087139260050502
3.0863383670488136
3.0854827497892905
3.084572381662069
3.0836070

KeyboardInterrupt: 